In [26]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='./data/ENZYMES', name='ENZYMES')

In [27]:
len(dataset)

600

In [29]:
dataset.num_classes

6

In [30]:
dataset.num_node_features

3

In [31]:
data = dataset[0]
print(data)

Data(edge_index=[2, 168], x=[37, 3], y=[1])


In [32]:
data.is_undirected()

True

In [33]:
# Making training data, testing data
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [34]:
# Shuffling data before split
dataset = dataset.shuffle()

# This is equivalent of doing:
import torch
perm  = torch.randperm(len(dataset))
dataset = dataset[perm]

In [35]:
# Cora dataset
# for semi-supervised graph node classification

from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./data/Cora', name='Cora')


In [36]:
print(len(dataset))

1


In [37]:
dataset.num_classes

7

In [38]:
dataset.num_node_features

1433

In [39]:
# Single, undirected citation graph
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [40]:
# train_mask -> denotes which nodes to train
# val_mask -> denotes which nodes to use for validation
# test_mask -> denotes against which nodes to test

print(data.is_undirected())
print(data.train_mask.sum().item())
print(data.test_mask.sum().item())

True
140
1000


In [41]:
# Mini batches
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='./data/ENZYMES', name="ENZYMES", use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)

DataBatch(edge_index=[2, 3994], x=[1001, 21], y=[32], batch=[1001], ptr=[33])
32
DataBatch(edge_index=[2, 3808], x=[987, 21], y=[32], batch=[987], ptr=[33])
32
DataBatch(edge_index=[2, 3740], x=[986, 21], y=[32], batch=[986], ptr=[33])
32
DataBatch(edge_index=[2, 3766], x=[1003, 21], y=[32], batch=[1003], ptr=[33])
32
DataBatch(edge_index=[2, 4228], x=[1290, 21], y=[32], batch=[1290], ptr=[33])
32
DataBatch(edge_index=[2, 4082], x=[1067, 21], y=[32], batch=[1067], ptr=[33])
32
DataBatch(edge_index=[2, 4156], x=[1097, 21], y=[32], batch=[1097], ptr=[33])
32
DataBatch(edge_index=[2, 4044], x=[1055, 21], y=[32], batch=[1055], ptr=[33])
32
DataBatch(edge_index=[2, 4176], x=[1065, 21], y=[32], batch=[1065], ptr=[33])
32
DataBatch(edge_index=[2, 4216], x=[1066, 21], y=[32], batch=[1066], ptr=[33])
32
DataBatch(edge_index=[2, 3792], x=[989, 21], y=[32], batch=[989], ptr=[33])
32
DataBatch(edge_index=[2, 3954], x=[1025, 21], y=[32], batch=[1025], ptr=[33])
32
DataBatch(edge_index=[2, 4136], x=

Learning Methods on Graphs

In [44]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./data/Cora', name="Cora")

In [45]:
# Implementing two-layer GCN

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [46]:
# Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# print(dataset[2])

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [47]:
#Evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7980


In [48]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [49]:
print(dataset.num_classes)

7
